In [1]:
# SEKCJA 1: Import bibliotek
import pandas as pd
import numpy as np
import random
from datetime import datetime

print("✅ Biblioteki załadowane!")
print(f"📅 Data: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Biblioteki załadowane!
📅 Data: 2025-10-05 18:08


In [2]:
# SEKCJA 2: Specyfikacje techniczne 10 modeli Atlas
# Dane rzeczywiste z dokumentacji

material_handlers_specs = pd.DataFrame([
    # Model, Waga(t), Moc(kW), Zasięg(m), Cena nowa(zł), Hybrydowa
    ['160MH', 16.1, 105, 9.7, 480000, False],
    ['180MH', 21.1, 105, 11.6, 530000, False],
    ['200MH', 21.9, 129, 11.25, 570000, False],
    ['200MH ACCU', 21.7, 140, 11.6, 620000, True],  # Hybrydowa!
    ['250MH', 25.9, 129, 12.25, 680000, False],
    ['270MH', 30.8, 129, 15.31, 750000, False],
    ['300MH', 33.2, 129, 15.31, 820000, False],
    ['350MH', 38.0, 180, 18.0, 950000, False],
    ['400MH', 44.5, 180, 17.9, 1100000, False],
    ['520MH', 57.0, 230, 21.9, 1350000, False]
], columns=['model', 'tonnage', 'engine_power_kw', 'reach_m', 'base_price_new', 'is_hybrid'])

print("📋 SPECYFIKACJE 10 MODELI ATLAS:")
print("="*70)
print(material_handlers_specs.to_string(index=False))
print("="*70)
print(f"✅ {len(material_handlers_specs)} modeli załadowanych")

📋 SPECYFIKACJE 10 MODELI ATLAS:
     model  tonnage  engine_power_kw  reach_m  base_price_new  is_hybrid
     160MH     16.1              105     9.70          480000      False
     180MH     21.1              105    11.60          530000      False
     200MH     21.9              129    11.25          570000      False
200MH ACCU     21.7              140    11.60          620000       True
     250MH     25.9              129    12.25          680000      False
     270MH     30.8              129    15.31          750000      False
     300MH     33.2              129    15.31          820000      False
     350MH     38.0              180    18.00          950000      False
     400MH     44.5              180    17.90         1100000      False
     520MH     57.0              230    21.90         1350000      False
✅ 10 modeli załadowanych


In [3]:
# SEKCJA 3: Funkcja generująca realistyczne maszyny używane (OSTATECZNA)

def generate_material_handler_data(n_observations=500, current_year=2025):
    """
    Generuje n_observations realistycznych maszyn używanych
    
    WAŻNE: 
    - Zapisujemy tylko YEAR (rok produkcji)
    - AGE jest obliczany automatycznie gdy potrzeba
    - Model będzie trenowany z AGE (obliczonym z YEAR)
    """
    
    data = []
    
    # Stany techniczne
    conditions = ['Doskonały', 'Bardzo dobry', 'Dobry', 'Zadowalający', 'Wymaga naprawy']
    condition_multipliers = {
        'Doskonały': 1.10,
        'Bardzo dobry': 1.00,
        'Dobry': 0.90,
        'Zadowalający': 0.75,
        'Wymaga naprawy': 0.60
    }
    
    # Dla każdego modelu wygeneruj około n/10 obserwacji
    for idx, spec in material_handlers_specs.iterrows():
        model_name = spec['model']
        tonnage = spec['tonnage']
        power = spec['engine_power_kw']
        reach = spec['reach_m']
        base_price = spec['base_price_new']
        is_hybrid = spec['is_hybrid']
        
        # Liczba obserwacji dla tego modelu (50-60)
        n_for_model = random.randint(50, 60)
        
        for i in range(n_for_model):
            # ✅ Generujemy ROK PRODUKCJI (2010-2024)
            year = random.randint(2010, 2024)
            
            # ✅ Obliczamy wiek (do użycia w logice, ale NIE zapisujemy)
            age = current_year - year
            
            # Motogodziny (zależne od wieku)
            avg_hours_per_year = random.uniform(800, 1500)
            hours = int(age * avg_hours_per_year * random.uniform(0.7, 1.3))
            hours = max(0, min(hours, 25000))
            
            # Stan techniczny (gorszy dla starszych i bardziej używanych)
            condition_score = 4
            condition_score -= min(2, age // 4)
            condition_score -= min(2, hours // 8000)
            condition_score += random.randint(-1, 1)
            condition_score = max(0, min(4, condition_score))
            condition = conditions[condition_score]
            
            # Serwis
            full_service_history = random.random() > 0.3
            major_repairs = max(0, min(5, int(age / 3) + random.randint(-1, 2)))
            
            # Wyposażenie
            has_ac = random.random() > 0.25
            has_gps = year >= 2015 and random.random() > 0.4  # GPS w nowszych
            extra_attachments = random.randint(0, 3)
            
            # KALKULACJA CENY (używamy age w obliczeniach)
            price = base_price
            price *= 1.15  # Brand premium
            price *= (0.91 ** age)  # Deprecjacja przez wiek
            price *= max(0.6, 1 - (hours / 50000) * 0.4)  # Deprecjacja przez motogodziny
            price *= condition_multipliers[condition]  # Stan
            
            if full_service_history:
                price *= 1.05
            price *= (1 - major_repairs * 0.03)
            
            if has_ac:
                price *= 1.05
            if has_gps:
                price *= 1.03
            price += extra_attachments * 8000
            
            if is_hybrid:
                price *= 1.05
            
            price *= random.uniform(0.90, 1.10)  # Szum
            price = int(round(price / 1000) * 1000)
            
            # ✅ ZAPISUJEMY: Tylko YEAR (nie age)
            data.append({
                'model': model_name,
                'brand': 'Atlas',
                'tonnage': tonnage,
                'engine_power_kw': power,
                'reach_m': reach,
                'base_price_new': base_price,
                'is_hybrid': is_hybrid,
                'year': year,  # ✅ TYLKO YEAR
                'hours': hours,
                'hours_per_year': round(hours / age if age > 0 else 0, 1),
                'condition': condition,
                'condition_score': condition_score,
                'full_service_history': full_service_history,
                'major_repairs': major_repairs,
                'has_ac': has_ac,
                'has_gps': has_gps,
                'extra_attachments': extra_attachments,
                'price': price
            })
    
    return pd.DataFrame(data)

print("✅ Funkcja generująca gotowa! (OSTATECZNA WERSJA)")
print("📝 Zapisujemy: YEAR | Model używa: AGE (obliczone automatycznie)")

✅ Funkcja generująca gotowa! (OSTATECZNA WERSJA)
📝 Zapisujemy: YEAR | Model używa: AGE (obliczone automatycznie)


In [4]:
# SEKCJA 4: Wygeneruj 500 obserwacji

print("🔄 Generowanie danych...")
df = generate_material_handler_data(n_observations=500)

# ✅ Oblicz AGE do wyświetlenia statystyk (ale nie zapisujemy do CSV!)
current_year = 2025
df_with_age = df.copy()
df_with_age['age'] = current_year - df_with_age['year']

print(f"✅ Wygenerowano {len(df)} maszyn!")
print("\n📊 STATYSTYKI:")
print("="*70)
print(f"Najstarszy rok:  {df['year'].min()}")
print(f"Najnowszy rok:   {df['year'].max()}")
print(f"Zakres wieku:    {df_with_age['age'].min()} - {df_with_age['age'].max()} lat")
print(f"Min motogodziny: {df['hours'].min():,} godz")
print(f"Max motogodziny: {df['hours'].max():,} godz")
print(f"Najtańsza: {df['price'].min():,} zł")
print(f"Najdroższa: {df['price'].max():,} zł")
print(f"Średnia cena: {int(df['price'].mean()):,} zł")
print("="*70)

print("\n📋 ROZKŁAD WG MODELI:")
print(df['model'].value_counts().sort_index())

print("\n📋 ROZKŁAD WG STANU:")
print(df['condition'].value_counts())

print("\n📋 ROZKŁAD WG ROKU:")
print(df['year'].value_counts().sort_index())

print("\n🔍 PRZYKŁADOWE 5 MASZYN:")
print(df_with_age[['model', 'year', 'age', 'hours', 'condition', 'price']].head())

print("\n✅ STRUKTURA DANYCH:")
print(f"   Zapisane kolumny: {list(df.columns)}")
print(f"   'year' w danych: {'year' in df.columns} ✅")
print(f"   'age' w danych: {'age' in df.columns} ❌ (obliczamy automatycznie)")

🔄 Generowanie danych...
✅ Wygenerowano 543 maszyn!

📊 STATYSTYKI:
Najstarszy rok:  2010
Najnowszy rok:   2024
Zakres wieku:    1 - 15 lat
Min motogodziny: 745 godz
Max motogodziny: 24,705 godz
Najtańsza: 90,000 zł
Najdroższa: 1,152,000 zł
Średnia cena: 364,902 zł

📋 ROZKŁAD WG MODELI:
model
160MH         56
180MH         55
200MH         51
200MH ACCU    57
250MH         50
270MH         59
300MH         51
350MH         53
400MH         60
520MH         51
Name: count, dtype: int64

📋 ROZKŁAD WG STANU:
condition
Doskonały         112
Dobry             110
Bardzo dobry      110
Wymaga naprawy    109
Zadowalający      102
Name: count, dtype: int64

📋 ROZKŁAD WG ROKU:
year
2010    29
2011    38
2012    33
2013    37
2014    37
2015    45
2016    46
2017    32
2018    27
2019    34
2020    36
2021    32
2022    38
2023    40
2024    39
Name: count, dtype: int64

🔍 PRZYKŁADOWE 5 MASZYN:
   model  year  age  hours     condition   price
0  160MH  2021    4   3064         Dobry  343000
1  160

In [5]:
# SEKCJA 5: Zapisz dane do CSV (tylko z YEAR, bez AGE)

# Zapisz specyfikacje modeli
specs_path = '../data/material_handlers_specs.csv'
material_handlers_specs.to_csv(specs_path, index=False, encoding='utf-8')
print(f"✅ Zapisano specyfikacje: {specs_path}")

# ✅ Zapisz dane treningowe (BEZ kolumny age - będzie obliczona automatycznie)
training_path = '../data/material_handlers_training.csv'
df.to_csv(training_path, index=False, encoding='utf-8')
print(f"✅ Zapisano dane treningowe: {training_path}")

print("\n📁 PLIKI GOTOWE:")
print(f"  1. {specs_path} ({len(material_handlers_specs)} modeli)")
print(f"  2. {training_path} ({len(df)} obserwacji)")

print(f"\n✅ KOLUMNY W CSV: {list(df.columns)}")
print("\n💡 ZASADA:")
print("   • W CSV: zapisujemy 'year' (2018)")
print("   • W modelu: używamy 'age' (obliczamy automatycznie: 2025 - year)")
print("   • W aplikacji: handlowiec wpisuje 'year', obliczamy 'age'")
print("   • JEDNA funkcja calculate_age() używana wszędzie - zero błędów!")


✅ Zapisano specyfikacje: ../data/material_handlers_specs.csv
✅ Zapisano dane treningowe: ../data/material_handlers_training.csv

📁 PLIKI GOTOWE:
  1. ../data/material_handlers_specs.csv (10 modeli)
  2. ../data/material_handlers_training.csv (543 obserwacji)

✅ KOLUMNY W CSV: ['model', 'brand', 'tonnage', 'engine_power_kw', 'reach_m', 'base_price_new', 'is_hybrid', 'year', 'hours', 'hours_per_year', 'condition', 'condition_score', 'full_service_history', 'major_repairs', 'has_ac', 'has_gps', 'extra_attachments', 'price']

💡 ZASADA:
   • W CSV: zapisujemy 'year' (2018)
   • W modelu: używamy 'age' (obliczamy automatycznie: 2025 - year)
   • W aplikacji: handlowiec wpisuje 'year', obliczamy 'age'
   • JEDNA funkcja calculate_age() używana wszędzie - zero błędów!
